Start by loading the PETLION package

In [1]:
using PETLION

Let's see how long it takes to create the model for an isothermal, LCO battery

In [2]:
@time p = Params(
    LCO;
    N_p = 10, # discretizations in the cathode
    N_s = 10, # discretizations in the separator
    N_n = 10, # discretizations in the anode
    N_r_p = 10, # discretizations in the solid cathode particles
    N_r_n = 10, # discretizations in the solid anode particles
    temperature = false # temperature enabled or disabled
)

Creating the functions for param_skeleton:
  Cathode: LCO, rxn_BV, & OCV_LCO
  Anode:   LiC6, rxn_BV, & OCV_LiC6
  System:  D_s_eff, rxn_rate, D_eff_linear, K_eff, & thermodynamic_factor_linear
  --------
  Temperature:     false
  Solid diffusion: Fickian, finite_difference
  Aging:           false
  Voltage bounds:  [2.5 V, 4.3 V]
  SOC bounds:      [0.0, 1.0]
  --------
  N.p:   10
  N.r_p: 10
  N.s:   10
  N.n:   10
  N.r_n: 10

May take a few minutes...
Making initial guess function
Making symbolic model
Making symbolic Jacobian
Making initial condition functions
Finished

108.473750 seconds (241.09 M allocations: 13.017 GiB, 4.23% gc time, 3.05% compilation time)


param{jacobian_symbolic}:
  Cathode: LCO, rxn_BV, & OCV_LCO
  Anode:   LiC6, rxn_BV, & OCV_LiC6
  System:  D_s_eff, rxn_rate, D_eff_linear, K_eff, & thermodynamic_factor_linear
  --------
  Temperature:     false
  Solid diffusion: Fickian, finite_difference
  Aging:           false
  Voltage bounds:  [2.5 V, 4.3 V]
  SOC bounds:      [0.0, 1.0]
  --------
  N.p:   10
  N.r_p: 10
  N.s:   10
  N.n:   10
  N.r_n: 10

Quite awhile! The good news is we only need to do this once, every other time the model is loaded from the folder `saved_models` in your working directory which is much faster.

Now, let's try running a 1C discharge starting from full capacity.

In [3]:
@time model = run_model(p,I=-1,SOC=1)

 66.724029 seconds (176.40 M allocations: 28.151 GiB, 20.05% gc time)


PETLION model
  --------
  Run:     I
  Time:    3604.24 s
  Voltage: 2.8687 V
  Current: -1C
  Power:   -83.8509 W
  SOC:     0.0
  Exit:    Below minimum SOC limit

Since Julia is a compiled language, the first run of a function is going to be a little slow. If we run it a few more times we will see its real speed.

In [4]:
@time model = run_model(p,I=-1,SOC=1)
@time model = run_model(p,I=-1,SOC=1)
@time model = run_model(p,I=-1,SOC=1)
@time model = run_model(p,I=-1,SOC=1)

  0.039930 seconds (4.95 k allocations: 1.254 MiB, 88.70% compilation time)
  0.003514 seconds (1.18 k allocations: 1.065 MiB)
  0.003486 seconds (1.18 k allocations: 1.065 MiB)
  0.003406 seconds (1.18 k allocations: 1.065 MiB)


PETLION model
  --------
  Run:     I
  Time:    3604.24 s
  Voltage: 2.8687 V
  Current: -1C
  Power:   -83.8509 W
  SOC:     0.0
  Exit:    Below minimum SOC limit

Now we're seeing the performance we want! Let's use the package `BenchmarkTools` to get a more in-depth view of the performance

In [5]:
using BenchmarkTools
@benchmark $run_model($p,I=$(-1),SOC=$1)

BenchmarkTools.Trial: 1632 samples with 1 evaluation.
 Range (min … max):  2.521 ms …   9.395 ms  ┊ GC (min … max): 0.00% … 63.07%
 Time  (median):     2.832 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.056 ms ± 696.455 μs  ┊ GC (mean ± σ):  1.83% ±  6.01%

    █▂                                                         
  ▄▅██▆▅▄▃▅▅▄▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂ ▂
  2.52 ms         Histogram: frequency by time        7.73 ms <

 Memory estimate: 1.07 MiB, allocs estimate: 1175.

We see a runtime of 3.1 ms averaged over 1632 runs.